In [1]:
import sys; sys.path.insert(0, "../"); from utils import *

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from sympy import sympify
import sys; sys.path.insert(0, "../"); from utils import *
from parametric_discovery_pinn import ParametricPINN, BurgerPINN
from madgrad import MADGRAD

from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from robust_pde_diff import print_pde, RobustPCA, Robust_LRSTR
from parametric_pde_diff import TrainSGTRidge, create_groups

from scipy.integrate import odeint
from numpy.fft import fft, ifft, fftfreq
from time import time

from pysr import pysr, best

fontsize = 20

In [3]:
data = pickle_load("../parametric_pde_data/parametric_burgers.pkl")

x = data['x']; spatial_dims = x.shape[0]
t = data['t']; time_dims = t.shape[0]

Exact = data['u']
X, T = np.meshgrid(x, t)

X_star = np.hstack((to_column_vector(X), to_column_vector(T)))
u_star = to_column_vector(Exact.T)

# domain bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

# Sampling training data points
N = 20000
training_idxs = sampling_from_rows(X_star, N, True)
X_train = X_star[training_idxs, :]
u_train = u_star[training_idxs, :]

# to_tensor
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)
X_train = to_tensor(X_train, True)
u_train = to_tensor(u_train, False)
lb = to_tensor(lb, False)
ub = to_tensor(ub, False)

u_xx_true = 0.1*np.ones(time_dims)
uu_x_true = -1*(1+0.25*np.sin(t))

Loaded from ../parametric_pde_data/parametric_burgers.pkl


In [4]:
pinn = ParametricPINN(scale=False, lb=lb, ub=ub)
print("Loaded the pretrained weights")
pinn.load_state_dict(torch.load("./saved_path_inverse_parametric_burgers/parametric_pinn.pth"))

Loaded the pretrained weights


<All keys matched successfully>

In [5]:
expression, _ = build_exp("-0.1872898*sin(t)-1.0238724")
mod = SymPyModule(expressions=[expression])

Please ensure that all coefs are different


In [6]:
model = nn.Sequential(pinn.preprocessor_net, pinn.pde_solver_net)
pinn = BurgerPINN(model=model, funcs=mod, scale=False, lb=lb, ub=ub)

In [7]:
def pcgrad_closure(return_list=False):
    global N, X_train, u_train
    losses = pinn.loss(*dimension_slicing(X_train), u_train)
    updated_grads = []
    
    for i in range(2):
        optimizer.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in pinn.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(pinn.parameters()):
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    if not return_list: return losses[0]+losses[1]
    else: return losses

In [8]:
def finetuning_closure():
    global N, X_train, u_train
    if torch.is_grad_enabled(): f_opt.zero_grad()
    # the solver network only consider the first N samples.
    mse_loss, pde_loss = pinn.loss(*dimension_slicing(X_train), u_train)
    loss = mse_loss + pde_loss
    if loss.requires_grad: loss.backward(retain_graph=False)
    return loss

In [9]:
optimizer = MADGRAD(pinn.parameters(), lr=1e-5, momentum=0.9)
for i in range(150):
    pinn.train()
    optimizer.step(pcgrad_closure)
    if i%10==0:
        loss = pcgrad_closure(return_list=True)
        print(loss)
        
f_opt = torch.optim.LBFGS(pinn.parameters(), lr=1e-1, max_iter=300, max_eval=int(1.25*300), history_size=300, line_search_fn='strong_wolfe')
for i in range(100):
    f_opt.step(finetuning_closure)
    if i%10==0:
        loss = finetuning_closure()
        print(loss.item())

(tensor(1.4728e-06, grad_fn=<MseLossBackward>), tensor(1.6235e-05, grad_fn=<MseLossBackward>))
(tensor(1.5480e-06, grad_fn=<MseLossBackward>), tensor(1.5724e-05, grad_fn=<MseLossBackward>))
(tensor(1.5743e-06, grad_fn=<MseLossBackward>), tensor(1.5097e-05, grad_fn=<MseLossBackward>))
(tensor(1.5908e-06, grad_fn=<MseLossBackward>), tensor(1.4492e-05, grad_fn=<MseLossBackward>))
(tensor(1.6750e-06, grad_fn=<MseLossBackward>), tensor(1.3866e-05, grad_fn=<MseLossBackward>))
(tensor(1.7436e-06, grad_fn=<MseLossBackward>), tensor(1.3289e-05, grad_fn=<MseLossBackward>))
(tensor(1.8061e-06, grad_fn=<MseLossBackward>), tensor(1.2756e-05, grad_fn=<MseLossBackward>))
(tensor(1.8518e-06, grad_fn=<MseLossBackward>), tensor(1.2273e-05, grad_fn=<MseLossBackward>))
(tensor(1.9114e-06, grad_fn=<MseLossBackward>), tensor(1.1814e-05, grad_fn=<MseLossBackward>))
(tensor(1.9878e-06, grad_fn=<MseLossBackward>), tensor(1.1376e-05, grad_fn=<MseLossBackward>))
(tensor(2.0668e-06, grad_fn=<MseLossBackward>), te

In [10]:
pinn.epsilon

Parameter containing:
tensor(0.0998, requires_grad=True)

In [11]:
[x.item() for x in pinn.funcs.parameters()]

[-1.000081181526184, -0.24583321809768677]

In [12]:
errs = np.array([100*(0.1-0.0998), 100*(1.000081181526184-1), 100*(0.25-0.24583321809768677)/0.25])
errs.mean(), errs.std()

(0.5649436378479006, 0.7790835192709206)

In [13]:
# torch.save(pinn.state_dict(), "./saved_path_inverse_parametric_burgers/final_finetuned_pinn.pth")